# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Proyecto Integrador**

### **Tecnológico de Monterrey**

### **Prof. Dra. Grettel Barceló Alonso**

### EQUIPO 20

- ### OSCAR MAURICIO BECERRA ALEGRÍA | A01795611

- ### VÍCTOR DANIEL BOHÓRQUEZ TORIBIO | A01794554

- ### ALAN JASSO ARENAS | A01383272



## **Ingeniería de Características para modelos de Machine Learning Clásicos.**

###**Proyecto de análisis y pronóstico epidemiológico de enfermedades neurológicas y trastornos mentales en México, con énfasis en la enfermedad de Parkinson (EP) durante 2014–2024**

En este notebook se presenta la ingeniería de características para obtener nuestro dataset para entrenar modelos de Machine Learning con nuestro problema.

Para ellos utilizaremos nuestro dataset antes de las variables exógenas.

# Lectura del datset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Cargamos el archivo de Google Drive en un DataFrame de pandas
ruta = "./data_parkinson.xlsx"
data_parkinson = pd.read_excel(ruta)

D:\Maestria\Proyecto Final\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
import unicodedata
pd.options.display.float_format = "{:,.2f}".format

In [4]:
# Normalización básica
data_parkinson.columns = data_parkinson.columns.str.strip()
for c in data_parkinson.select_dtypes(include='object').columns:
    data_parkinson[c] = data_parkinson[c].astype(str).str.strip()

# Tipos numéricos en Año
data_parkinson["Año"] = pd.to_numeric(data_parkinson["Año"], errors="coerce").astype("Int64")

# Convertir Valor a numérico y reemplazar nulos por 0
if "Valor" in data_parkinson.columns:
    data_parkinson["Valor"] = pd.to_numeric(data_parkinson["Valor"], errors="coerce").fillna(0).astype(int)

In [5]:
# Limpiar nombres de columnas
data_parkinson.columns = data_parkinson.columns.str.strip()

# Eliminar columnas innecesarias
cols_posibles = ['Pag.', 'Cuadro', 'No_01', 'No_02', 'No_03', 'No_04', 'Ax_002', 'Ax_001']
cols_drop = [c for c in cols_posibles if c in data_parkinson.columns]
data_parkinson = data_parkinson.drop(columns=cols_drop)

# Renombrar columnas si existen
rename_map = {}
if 'Ax_003' in data_parkinson.columns: rename_map['Ax_003'] = 'Sexo'
if 'Valor'  in data_parkinson.columns: rename_map['Valor']  = 'Casos'
if rename_map:
    data_parkinson = data_parkinson.rename(columns=rename_map)

# Eliminar filas con 'TOTAL' en Entidad y normalizar nombre
if 'Entidad' in data_parkinson.columns:
    data_parkinson["Entidad"] = data_parkinson["Entidad"].replace({"Distrito Federal": "Ciudad de México"})
    data_parkinson = data_parkinson[~data_parkinson["Entidad"].isin(["TOTAL"])]

# Normaliza 'Sexo'
if 'Sexo' in data_parkinson.columns:
    data_parkinson["Sexo"] = (data_parkinson["Sexo"].astype(str)
                              .str.strip()
                              .str.replace(r"\.", "", regex=True)
                              .str.replace(r"\s+", "", regex=True)
                              .str.upper())

# Limpiar y estandarizar el texto en la columna Padecimiento
def normalizar_texto(texto: str):
    if pd.isna(texto): return texto
    texto = str(texto).replace("\n", " ")
    texto = " ".join(texto.split())
    texto = texto.replace("CIE", " CIE")
    texto = unicodedata.normalize("NFKC", texto)
    return texto.strip()

if 'Padecimiento' in data_parkinson.columns:
    data_parkinson["Padecimiento"] = data_parkinson["Padecimiento"].apply(normalizar_texto)

# --- FILTRO: trabajar solo con Parkinson
if 'Padecimiento' in data_parkinson.columns:
    pat = r"(?i)\bparkinson\b|\bG20\b"
    n0 = len(data_parkinson)
    data_parkinson = data_parkinson[data_parkinson["Padecimiento"].str.contains(pat, na=False)].copy()
    print(f"Filas totales antes: {n0}  |  Parkinson después del filtro: {len(data_parkinson)}")
    print("Padecimientos únicos (post-filtro):",
          sorted(data_parkinson['Padecimiento'].dropna().unique().tolist())[:10])

# Casos a numérico
if 'Casos' in data_parkinson.columns:
    data_parkinson["Casos"] = pd.to_numeric(data_parkinson["Casos"], errors="coerce")

Filas totales antes: 319648  |  Parkinson después del filtro: 71520
Padecimientos únicos (post-filtro): ['Enfermedad de Parkinson  CIE-10a REV. G20']


In [6]:
from datetime import date, timedelta

df = data_parkinson.copy()

# Normaliza Sexo y filtra
df["Sexo_norm"] = (
    df["Sexo"].astype(str).str.strip()
      .str.replace(r"\.$", "", regex=True)
      .str.replace(r"\s+", "", regex=True)
      .str.upper()
)
df = df[df["Sexo_norm"].isin(["SEM", "H", "M"])].copy()

# Asegurar 'Casos' numérico
df["Casos"] = pd.to_numeric(df.get("Casos"), errors="coerce")

# Extraer número de semana como entero
if "Semana" in df.columns:
    df["Semana_num"] = (
        df["Semana"].astype(str).str.extract(r"(\d+)").astype(float).astype("Int64")
    )
else:
    df["Semana_num"] = pd.Series(pd.NA, index=df.index, dtype="Int64")

# Pivot (usando Sexo_norm)

df_pivot = (
    df.pivot_table(
        index=["Año", "Semana_num", "Entidad", "Padecimiento"],
        columns="Sexo_norm",
        values="Casos",
        aggfunc="sum"
    )
    .reset_index()
)

# Renombrar métricas y quitar la cabecera
rename_cols = {}
if "SEM" in df_pivot.columns: rename_cols["SEM"] = "Nuevos_Casos"
if "H"   in df_pivot.columns: rename_cols["H"]   = "Acum_H"
if "M"   in df_pivot.columns: rename_cols["M"]   = "Acum_M"
df_pivot = df_pivot.rename(columns=rename_cols)
df_pivot.columns.name = None

# Calendario domingo–sábado reglas del boletín
def mmwr_calendar_wed_rule(years):

    years = sorted(set(int(y) for y in years if pd.notna(y)))
    if not years:
        return pd.DataFrame(columns=["Año","Semana_num","Fecha_inicio","Fecha_fin"])
    y_min, y_max = min(years), max(years)

    # Genera domingos consecutivos incluyendo cierre del año anterior
    d = date(y_min - 1, 12, 31)
    d -= timedelta(days=(d.weekday() + 1) % 7)  # retrocede hasta domingo (Mon=0..Sun=6)

    rows = []
    while True:
        start_sun = d
        end_sat   = d + timedelta(days=6)
        wed       = d + timedelta(days=3)
        week_year = wed.year
        rows.append((week_year, start_sun, end_sat))
        d += timedelta(days=7)
        if d.year > y_max + 1:
            break

    cal = pd.DataFrame(rows, columns=["Año","Fecha_inicio","Fecha_fin"])
    cal = cal[cal["Año"].between(y_min, y_max)].copy()
    cal["Semana_num"] = cal.groupby("Año").cumcount() + 1
    return cal

# Construir calendario para los años presentes en el pivot
years_present = df_pivot["Año"].dropna().astype(int).unique()
cal_mmwr = mmwr_calendar_wed_rule(years_present)

# Reglas del boletín (última semana por año)
last_week = {
    2014:53, 2015:52, 2016:52, 2017:52, 2018:52,
    2019:52, 2020:53, 2021:52, 2022:52, 2023:52, 2024:52
}

cal_filt = cal_mmwr[cal_mmwr["Semana_num"] <= cal_mmwr["Año"].map(last_week)].copy()
# 2014 inicia en SE-2 (excluir SE-1)
cal_filt = cal_filt[~((cal_filt["Año"] == 2014) & (cal_filt["Semana_num"] == 1))].copy()


# Mantener semanas válidas y asignar fechas
df_pivot = df_pivot.merge(
    cal_filt[["Año","Semana_num","Fecha_inicio","Fecha_fin"]],
    on=["Año","Semana_num"], how="inner"
)

# Fecha = domingo de la semana
df_pivot["Fecha"] = pd.to_datetime(df_pivot["Fecha_inicio"]).dt.normalize()

# Orden, deduplicación y tipos
df_pivot = df_pivot.sort_values(["Entidad", "Padecimiento", "Fecha"]).reset_index(drop=True)
metricas = [c for c in ["Nuevos_Casos", "Acum_H", "Acum_M"] if c in df_pivot.columns]

if df_pivot.duplicated(["Entidad", "Padecimiento", "Fecha"]).any():
    df_pivot = (
        df_pivot
        .groupby(["Entidad", "Padecimiento", "Fecha"], as_index=False)[metricas]
        .sum()
    )

for c in metricas:
    df_pivot[c] = pd.to_numeric(df_pivot[c], errors="coerce")

# Reordenar columnas
cols = ["Año", "Semana_num", "Fecha", "Fecha_inicio", "Fecha_fin",
        "Entidad", "Padecimiento"] + metricas
df_pivot = df_pivot[[c for c in cols if c in df_pivot.columns]]

In [7]:
df_pivot

,Año,Semana_num,Fecha,Fecha_inicio,Fecha_fin,Entidad,Padecimiento,Nuevos_Casos,Acum_H,Acum_M
0,2014,2,2014-01-05,2014-01-05,2014-01-11,Aguascalientes,Enfermedad de Parkinson CIE-10a REV. G20,0,0,0
1,2014,3,2014-01-12,2014-01-12,2014-01-18,Aguascalientes,Enfermedad de Parkinson CIE-10a REV. G20,0,0,0
2,2014,4,2014-01-19,2014-01-19,2014-01-25,Aguascalientes,Enfermedad de Parkinson CIE-10a REV. G20,1,0,1
3,2014,5,2014-01-26,2014-01-26,2014-02-01,Aguascalientes,Enfermedad de Parkinson CIE-10a REV. G20,0,0,1
4,2014,6,2014-02-02,2014-02-02,2014-02-08,Aguascalientes,Enfermedad de Parkinson CIE-10a REV. G20,0,0,1
...,...,...,...,...,...,...,...,...,...,...
18299,2024,48,2024-11-24,2024-11-24,2024-11-30,Zacatecas,Enfermedad de Parkinson CIE-10a REV. G20,0,12,10
18300,2024,49,2024-12-01,2024-12-01,2024-12-07,Zacatecas,Enfermedad de Parkinson CIE-10a REV. G20,0,12,10
18301,2024,50,2024-12-08,2024-12-08,2024-12-14,Zacatecas,Enfermedad de Parkinson CIE-10a REV. G20,0,12,10
18302,2024,51,2024-12-15,2024-12-15,2024-12-21,Zacatecas,Enfermedad de Parkinson CIE-10a REV. G20,1,13,10


# Imputación de Semanas Faltantes

In [8]:
from io import StringIO

# Tabla de registros
records_csv = """Aguascalientes,Enfermedad de Parkinson CIE-10a REV. G20,2,9,3
Baja California,Enfermedad de Parkinson CIE-10a REV. G20,5,19,15
Baja California Sur,Enfermedad de Parkinson CIE-10a REV. G20,2,8,7
Campeche,Enfermedad de Parkinson CIE-10a REV. G20,4,14,10
Coahuila,Enfermedad de Parkinson CIE-10a REV. G20,8,34,18
Colima,Enfermedad de Parkinson CIE-10a REV. G20,9,30,23
Chiapas,Enfermedad de Parkinson CIE-10a REV. G20,5,14,9
Chihuahua,Enfermedad de Parkinson CIE-10a REV. G20,11,61,42
Ciudad de México,Enfermedad de Parkinson CIE-10a REV. G20,5,41,32
Durango,Enfermedad de Parkinson CIE-10a REV. G20,6,36,33
Guanajuato,Enfermedad de Parkinson CIE-10a REV. G20,3,14,14
Guerrero,Enfermedad de Parkinson CIE-10a REV. G20,1,12,10
Hidalgo,Enfermedad de Parkinson CIE-10a REV. G20,3,6,9
Jalisco,Enfermedad de Parkinson CIE-10a REV. G20,16,73,65
México,Enfermedad de Parkinson CIE-10a REV. G20,5,36,35
Michoacán,Enfermedad de Parkinson CIE-10a REV. G20,13,32,43
Morelos,Enfermedad de Parkinson CIE-10a REV. G20,4,32,33
Nayarit,Enfermedad de Parkinson CIE-10a REV. G20,2,1,2
Nuevo León,Enfermedad de Parkinson CIE-10a REV. G20,6,25,27
Oaxaca,Enfermedad de Parkinson CIE-10a REV. G20,3,19,8
Puebla,Enfermedad de Parkinson CIE-10a REV. G20,1,14,10
Querétaro,Enfermedad de Parkinson CIE-10a REV. G20,4,9,6
Quintana Roo,Enfermedad de Parkinson CIE-10a REV. G20,1,7,5
San Luis Potosí,Enfermedad de Parkinson CIE-10a REV. G20,3,22,16
Sinaloa,Enfermedad de Parkinson CIE-10a REV. G20,15,44,45
Sonora,Enfermedad de Parkinson CIE-10a REV. G20,1,18,11
Tabasco,Enfermedad de Parkinson CIE-10a REV. G20,6,14,11
Tamaulipas,Enfermedad de Parkinson CIE-10a REV. G20,10,42,27
Tlaxcala,Enfermedad de Parkinson CIE-10a REV. G20,4,3,2
Veracruz,Enfermedad de Parkinson CIE-10a REV. G20,21,84,64
Yucatán,Enfermedad de Parkinson CIE-10a REV. G20,2,10,7
Zacatecas,Enfermedad de Parkinson CIE-10a REV. G20,0,3,8
"""

registros = pd.read_csv(
    StringIO(records_csv),
    header=None,
    names=["Entidad","Padecimiento","Nuevos_Casos","Acum_H","Acum_M"]
)

# Casteo numérico
for c in ["Nuevos_Casos","Acum_H","Acum_M"]:
    registros[c] = pd.to_numeric(registros[c], errors="coerce")

# Construir el "patch"
Y, W = 2024, 10
row = cal_filt.loc[(cal_filt["Año"]==Y) & (cal_filt["Semana_num"]==W)]
if row.empty:
    raise ValueError("SE 10-2024 no está en cal_filt. Revisa el calendario.")

fecha_ini = pd.to_datetime(row.iloc[0]["Fecha_inicio"])
fecha_fin = pd.to_datetime(row.iloc[0]["Fecha_fin"])

patch = registros.copy()
patch["Año"] = Y
patch["Semana_num"] = W
patch["Fecha_inicio"] = fecha_ini
patch["Fecha_fin"]    = fecha_fin
patch["Fecha"]        = fecha_ini
patch["Fuente_Patch"] = "valores_boletin_SE10_2024"

# Integrar en df_pivot y consolidar
metricas = [c for c in ["Nuevos_Casos","Acum_H","Acum_M"] if c in df_pivot.columns]

df_fixed = pd.concat([df_pivot, patch], ignore_index=True)

# Consolidar
if metricas:
    df_fixed = (df_fixed
        .groupby(["Año","Semana_num","Fecha","Entidad","Padecimiento"], as_index=False)[metricas]
        .sum()
    )
else:
    df_fixed = df_fixed.drop_duplicates(["Año","Semana_num","Fecha","Entidad","Padecimiento"])

# Reagregar rangos del calendario por si el groupby los quitó
df_fixed = df_fixed.merge(
    cal_filt[["Año","Semana_num","Fecha_inicio","Fecha_fin"]],
    on=["Año","Semana_num"], how="left"
).sort_values(["Entidad","Padecimiento","Fecha"]).reset_index(drop=True)

# Verificación rápida
fechas_esperadas = pd.DatetimeIndex(cal_filt["Fecha_inicio"].unique()).sort_values()
fechas_presentes = pd.DatetimeIndex(df_fixed["Fecha"].dropna().unique()).sort_values()
faltantes2 = fechas_esperadas.difference(fechas_presentes)

print("Faltantes globales tras imputar:", len(faltantes2))
print("Se encuentra el 2024-03-03:", pd.Timestamp("2024-03-03") in fechas_presentes)

# Muestra 5 filas de la semana imputada
display(df_fixed[(df_fixed["Año"]==Y) & (df_fixed["Semana_num"]==W)].head())

Faltantes globales tras imputar: 0
Se encuentra el 2024-03-03: True


,Año,Semana_num,Fecha,Entidad,Padecimiento,Nuevos_Casos,Acum_H,Acum_M,Fecha_inicio,Fecha_fin
572,2024,10,2024-03-03,Aguascalientes,Enfermedad de Parkinson CIE-10a REV. G20,2,9,3,2024-03-03,2024-03-09
1145,2024,10,2024-03-03,Baja California,Enfermedad de Parkinson CIE-10a REV. G20,5,19,15,2024-03-03,2024-03-09
1718,2024,10,2024-03-03,Baja California Sur,Enfermedad de Parkinson CIE-10a REV. G20,2,8,7,2024-03-03,2024-03-09
2291,2024,10,2024-03-03,Campeche,Enfermedad de Parkinson CIE-10a REV. G20,4,14,10,2024-03-03,2024-03-09
2864,2024,10,2024-03-03,Chiapas,Enfermedad de Parkinson CIE-10a REV. G20,5,14,9,2024-03-03,2024-03-09


# Manejo de Outliers

In [9]:
# Serie semanal nacional (usando dataset consolidado)
nac = (df_fixed.groupby('Fecha', as_index=False)['Nuevos_Casos']
         .sum()
         .sort_values('Fecha')
         .rename(columns={'Nuevos_Casos': 'Nuevos_NAC'}))

# Detección Hampel filter
def hampel_flags(x: pd.Series, window=9, k=5):
    """Detecta outliers con mediana móvil y MAD."""
    minp = max(3, window // 2)
    med = x.rolling(window, center=True, min_periods=minp).median()
    mad = (x - med).abs().rolling(window, center=True, min_periods=minp).median() * 1.4826
    flags = (mad > 0) & ((x - med).abs() > k * mad)
    return flags.fillna(False), med, mad

nac['is_outlier'], nac['mediana_rol'], nac['mad_rol'] = hampel_flags(
    nac['Nuevos_NAC'], window=9, k=5
)

# Calcular z para inspección
nac['z_robusto'] = (nac['Nuevos_NAC'] - nac['mediana_rol']) / nac['mad_rol']

# Mostrar semanas marcadas como outliers
outliers_semana = nac.loc[nac['is_outlier'], ['Fecha', 'Nuevos_NAC', 'mediana_rol', 'mad_rol', 'z_robusto']]
outliers_semana = outliers_semana.sort_values('Fecha').reset_index(drop=True)

print("OUTLIERS NACIONALES:", len(outliers_semana))
display(outliers_semana)

OUTLIERS NACIONALES: 10


,Fecha,Nuevos_NAC,mediana_rol,mad_rol,z_robusto
0,2016-03-27,66,160.00,14.83,-6.34
1,2016-05-15,1834,170.00,17.79,93.53
2,2016-09-18,103,161.00,10.38,-5.59
3,2017-11-05,115,151.00,5.93,-6.07
4,2018-12-30,92,151.00,5.93,-9.95
5,2019-07-28,131,158.00,2.97,-9.11
6,2019-11-24,136,162.00,4.45,-5.85
7,2020-09-20,45,64.00,1.48,-12.82
8,2022-09-18,98,143.00,4.45,-10.12
9,2023-04-09,98,150.00,7.41,-7.01


In [10]:
# PARÁMETROS
fecha_outlier = pd.Timestamp("2016-05-15")  # semana detectada
win = 9      # ventana centrada (~±4 semanas)
k_hist = 4   # semanas previas para fallback histórico
minp = max(3, win//2)

# PREPARACIÓN
DF = df_fixed.copy()
DF['Fecha'] = pd.to_datetime(DF['Fecha'])
DF = DF.sort_values(['Entidad','Fecha'])

# Serie nacional y mediana local nacional (target)
nac = (DF.groupby('Fecha', as_index=False)['Nuevos_Casos']
         .sum()
         .rename(columns={'Nuevos_Casos':'Nuevos_NAC'})
         .sort_values('Fecha'))

nac['mediana_rol'] = nac['Nuevos_NAC'].rolling(win, center=True, min_periods=minp).median()

nac_row = nac.loc[nac['Fecha']==fecha_outlier]
if nac_row.empty:
    raise ValueError("La fecha del outlier no existe en la serie nacional.")

raw_nac    = int(nac_row['Nuevos_NAC'].iloc[0])
target_nac = int(round(float(nac_row['mediana_rol'].iloc[0])))
exceso_nac = raw_nac - target_nac
if exceso_nac <= 0:
    raise ValueError(f"No hay exceso nacional que corregir en {fecha_outlier.date()} (exceso={exceso_nac}).")

print(f"[NACIONAL] {fecha_outlier.date()}  raw={raw_nac}  target={target_nac}  exceso={exceso_nac}")

# MEDIANA LOCAL POR ENTIDAD
DF['mediana_ent'] = (
    DF.sort_values(['Entidad','Fecha'])
      .groupby('Entidad')['Nuevos_Casos']
      .transform(lambda s: s.rolling(win, center=True, min_periods=minp).median())
)

# Tabla de semana (por entidad)
tab = DF.loc[DF['Fecha']==fecha_outlier, ['Entidad','Nuevos_Casos','mediana_ent']].copy()

# Fallback si hay bordes
if tab['mediana_ent'].isna().any():
    def mediana_vecina(ent):
        g = DF[DF['Entidad']==ent]
        win_mask = (g['Fecha']>=fecha_outlier - pd.Timedelta(days=28)) & \
                   (g['Fecha']<=fecha_outlier + pd.Timedelta(days=28)) & \
                   (g['Fecha']!=fecha_outlier)
        vals = g.loc[win_mask, 'Nuevos_Casos']
        return float(vals.median()) if len(vals) else float(g['Nuevos_Casos'].median())
    tab.loc[tab['mediana_ent'].isna(), 'mediana_ent'] = (
        tab.loc[tab['mediana_ent'].isna(), 'Entidad'].apply(mediana_vecina)
    )

# Exceso por entidad
tab['exceso_ent'] = (tab['Nuevos_Casos'] - tab['mediana_ent']).clip(lower=0)
sum_excesos = float(tab['exceso_ent'].sum())

# PARÁMETROS
fecha_outlier = pd.Timestamp("2016-05-15")  # semana detectada
win = 9      # ventana centrada (~±4 semanas)
k_hist = 4   # semanas previas para fallback histórico
minp = max(3, win//2)

# PREPARACIÓN
DF = df_fixed.copy()
DF['Fecha'] = pd.to_datetime(DF['Fecha'])
DF = DF.sort_values(['Entidad','Fecha'])

# Serie nacional y mediana local (target)
nac = (DF.groupby('Fecha', as_index=False)['Nuevos_Casos']
         .sum()
         .rename(columns={'Nuevos_Casos':'Nuevos_NAC'})
         .sort_values('Fecha'))

nac['mediana_rol'] = nac['Nuevos_NAC'].rolling(win, center=True, min_periods=minp).median()

nac_row = nac.loc[nac['Fecha']==fecha_outlier]
if nac_row.empty:
    raise ValueError("La fecha del outlier no existe en la serie nacional.")

raw_nac    = int(nac_row['Nuevos_NAC'].iloc[0])
target_nac = int(round(float(nac_row['mediana_rol'].iloc[0])))
exceso_nac = raw_nac - target_nac
if exceso_nac <= 0:
    raise ValueError(f"No hay exceso nacional que corregir en {fecha_outlier.date()} (exceso={exceso_nac}).")

print(f"[NACIONAL] {fecha_outlier.date()}  raw={raw_nac}  target={target_nac}  exceso={exceso_nac}")

# MEDIANA LOCAL POR ENTIDAD
DF['mediana_ent'] = (
    DF.sort_values(['Entidad','Fecha'])
      .groupby('Entidad')['Nuevos_Casos']
      .transform(lambda s: s.rolling(win, center=True, min_periods=minp).median())
)

# Tabla de la semana (por entidad)
tab = DF.loc[DF['Fecha']==fecha_outlier, ['Entidad','Nuevos_Casos','mediana_ent']].copy()

# Fallback si hay bordes
if tab['mediana_ent'].isna().any():
    def mediana_vecina(ent):
        g = DF[DF['Entidad']==ent]
        win_mask = (g['Fecha']>=fecha_outlier - pd.Timedelta(days=28)) & \
                   (g['Fecha']<=fecha_outlier + pd.Timedelta(days=28)) & \
                   (g['Fecha']!=fecha_outlier)
        vals = g.loc[win_mask, 'Nuevos_Casos']
        return float(vals.median()) if len(vals) else float(g['Nuevos_Casos'].median())
    tab.loc[tab['mediana_ent'].isna(), 'mediana_ent'] = (
        tab.loc[tab['mediana_ent'].isna(), 'Entidad'].apply(mediana_vecina)
    )

# Exceso por entidad
tab['exceso_ent'] = (tab['Nuevos_Casos'] - tab['mediana_ent']).clip(lower=0)
sum_excesos = float(tab['exceso_ent'].sum())

# PONDERACIÓN
if sum_excesos > 0:
    tab['peso'] = tab['exceso_ent'] / sum_excesos
else:
    hist = (DF[DF['Fecha'] < fecha_outlier]
              .sort_values('Fecha')
              .groupby('Entidad')
              .tail(k_hist)
              .groupby('Entidad')['Nuevos_Casos'].mean())
    hist = (hist / hist.sum()).reindex(tab['Entidad']).fillna(1.0/len(tab))
    tab['peso'] = hist.values

tab['ajuste'] = (exceso_nac * tab['peso']).round().astype(int)

# Ajuste de residuo por redondeo
residuo = exceso_nac - int(tab['ajuste'].sum())
if residuo != 0:
    idx_max = tab['ajuste'].idxmax()
    tab.loc[idx_max, 'ajuste'] += residuo

# APLICAR AJUSTE SOLO EN ESA SEMANA (en DF)
DF['Nuevos_Casos_raw'] = DF['Nuevos_Casos']
DF['Nuevos_Casos_fix'] = DF['Nuevos_Casos']  # columna auxiliar temporal

ajustes = tab.set_index('Entidad')['ajuste']
m = DF['Fecha'] == fecha_outlier

DF.loc[m, 'Nuevos_Casos_fix'] = (
    DF.loc[m].apply(
        lambda r: max(int(r['Nuevos_Casos']) - int(ajustes.get(r['Entidad'], 0)), 0),
        axis=1
    )
)

# Chequeo nacional inmediato (DF)
total_fix = int(DF.loc[DF['Fecha']==fecha_outlier, 'Nuevos_Casos_fix'].sum())
assert total_fix == target_nac, f"La suma corregida {total_fix} != target {target_nac}"
print("Suma nacional corregida coincide con el target.")

# Persistir en df_fixed
_keys = ['Fecha','Entidad','Padecimiento']

# Trae la corrección desde DF con nombre temporal y une
tmp = DF[_keys + ['Nuevos_Casos_fix']].rename(columns={'Nuevos_Casos_fix':'Nuevos_Casos_corr'})
df_fixed = df_fixed.merge(tmp, on=_keys, how='left')

# Coalesce
df_fixed['Nuevos_Casos'] = df_fixed['Nuevos_Casos_corr'].combine_first(df_fixed['Nuevos_Casos'])

# Limpieza de columnas auxiliares
df_fixed.drop(columns=['Nuevos_Casos_corr'], inplace=True, errors='ignore')
# Si en df_fixed existiera por accidente alguna *_fix pasada, elimínala:
df_fixed.drop(columns=['Nuevos_Casos_fix', 'Nuevos_Casos_raw', 'mediana_ent'], inplace=True, errors='ignore')

# Verificación nacional ya sobre df_fixed
total_fixed_df = int(df_fixed.loc[df_fixed['Fecha']==fecha_outlier, 'Nuevos_Casos'].sum())
assert total_fixed_df == target_nac, f"[df_fixed] total {total_fixed_df} != target {target_nac}"
print("Corrección persistida en df_fixed (solo 'Nuevos_Casos', sin columnas *_fix).")

[NACIONAL] 2016-05-15  raw=1834  target=170  exceso=1664
[NACIONAL] 2016-05-15  raw=1834  target=170  exceso=1664
Suma nacional corregida coincide con el target.
Corrección persistida en df_fixed (solo 'Nuevos_Casos', sin columnas *_fix).


In [11]:
# Definir el dataset maestro consolidado
df_final = df_fixed.copy()

# Confirmar estructura
print("Dataset consolidado listo:", df_final.shape)
print("Columnas:", df_final.columns.tolist())

Dataset consolidado listo: (18336, 10)
Columnas: ['Año', 'Semana_num', 'Fecha', 'Entidad', 'Padecimiento', 'Nuevos_Casos', 'Acum_H', 'Acum_M', 'Fecha_inicio', 'Fecha_fin']


# Ingeniería de Características

Eliminamos columnas que se utilizaron para la corrección y detección de outliers que son la fecha de inicio y fin.

In [12]:
df_final = df_final.drop(columns=["Fecha_inicio", "Fecha_fin"])

Debido a que nuestra predicción será sobre el número total de casos nuevos, y aún no sobre casos basados en el sexo, las variables de acumulados de hombre y mujer no nos sirven, así que las eliminamos del dataset.

In [13]:
df_final = df_final.drop(columns=["Acum_H", "Acum_M"])

Renombramos Semana_num a solo Semana para una mejor lectura.

In [14]:
df_final.rename(columns={"Semana_num": "Semana"}, inplace=True)

Para tener un dataset Con estimaciones nacionales, debemos agrupar nuestros datos por fecha, de esta manera podemos hacer la suma de dichas fechas, que a su vez filtra por año y semana. También descartamos la entidad, ya que los estados están incluidos en la nación, y por ahora no haremos predicciones estatales o regionales.

In [15]:
grouped = df_final.groupby(["Fecha", "Año", "Semana"])[["Nuevos_Casos"]].sum()
grouped = grouped.reset_index()
grouped

,Fecha,Año,Semana,Nuevos_Casos
0,2014-01-05,2014,2,11
1,2014-01-12,2014,3,89
2,2014-01-19,2014,4,127
3,2014-01-26,2014,5,114
4,2014-02-02,2014,6,159
...,...,...,...,...
568,2024-11-24,2024,48,116
569,2024-12-01,2024,49,153
570,2024-12-08,2024,50,132
571,2024-12-15,2024,51,139


Eliminamos la columna Fecha, que hasta ahora ha funcionado como un identificador único, así que no nos aporta nada a nuestros modelos.

In [16]:
grouped = grouped.drop(columns=["Fecha"])

## Añadimos varaibles cíclicas para las semanas. Utilizaremos Seno y Coseno.

Añadimos la variable cíclica de Seno para las semanas.

In [17]:
total_semanas = grouped["Semana"].max()
grouped["Semana_sin"] = np.sin(2 * np.pi * grouped["Semana"] / total_semanas)

Añadimos la varíable cíclica de Coseno para las semanas.

In [18]:
grouped["Semana_cos"] = np.cos(2 * np.pi * grouped["Semana"] / total_semanas)

## Variables de Media Móvil

Añadimos la media móvil del número de nuevos casos, para ello utilizaremos una ventana de 1, 3 y 12 meses (en semanas), la ventana de 1 mes tiene la función de encontrar eventos locales o eventos particulares, mientras que la ventana de 3 meses pretende tener una vision por cuarto del año, y finalmente los 12 meses pretenden demostrar un resumen tendencial del año completo.

Usamos un mínimo de periodos de 1 para evitar valores nulos en las primeras filas del dataset.

Añadimos la variable de la media móvil para 1 mes (4 semanas)

In [19]:
grouped["MM_Mensual"] = grouped["Nuevos_Casos"].rolling(window=4, center=True, min_periods=1).mean()

Añadimos la variable de media móvil para 3 meses (12 semanas)

In [20]:
grouped["MM_Cuarto"] = grouped["Nuevos_Casos"].rolling(window=12, center=True, min_periods=1).mean()

Añadimos la media movil anual (52 semanas)

In [21]:
grouped["MM_Anual"] = grouped["Nuevos_Casos"].rolling(window=52, center=True, min_periods=1).mean()

In [22]:
grouped

,Año,Semana,Nuevos_Casos,Semana_sin,Semana_cos,MM_Mensual,MM_Cuarto,MM_Anual
0,2014,2,11,0.23,0.97,50.00,103.33,112.12
1,2014,3,89,0.35,0.94,75.67,105.00,112.22
2,2014,4,127,0.46,0.89,85.25,106.62,112.79
3,2014,5,114,0.56,0.83,122.25,106.33,112.34
4,2014,6,159,0.65,0.76,130.00,107.70,111.93
...,...,...,...,...,...,...,...,...
568,2024,48,116,-0.56,0.83,140.00,140.18,142.16
569,2024,49,153,-0.46,0.89,137.00,138.40,141.13
570,2024,50,132,-0.35,0.94,135.00,137.44,140.38
571,2024,51,139,-0.23,0.97,131.00,135.88,141.18


## Variables de rezago o lag

Añadimos variables de rezago, retraso o lag, para dar a los modelos un contexto o influencia del pasado.

Usaremos distintos rezagos para probar con los modelos, hasta encontrar el mejor número de rezagos en relación a la importancia para el modelo.

In [23]:
grouped["lag_1"] = grouped["Nuevos_Casos"].shift(1)
grouped["lag_2"] = grouped["Nuevos_Casos"].shift(2)
grouped["lag_3"] = grouped["Nuevos_Casos"].shift(3)

## Codificación

En este caso, se va a codificar la variable Semana, si bien por el momento es númerica, también se va a codificar con OneHot ya que esta al estar en forma numérica podría indicar al modelo que la semana entre más alto el valor mejor, y esto no es así.

In [24]:
# One Hot Encoding
from sklearn.preprocessing import OneHotEncoder

encoder_sem = OneHotEncoder(sparse_output=False, drop= None)
one_hot_encoded = encoder_sem.fit_transform(grouped[["Semana"]])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder_sem.get_feature_names_out(["Semana"]))
df_encoded = pd.concat([grouped, one_hot_df], axis=1)
df_encoded = df_encoded.drop(["Semana"], axis=1)
df_encoded

,Año,Nuevos_Casos,Semana_sin,Semana_cos,MM_Mensual,MM_Cuarto,MM_Anual,lag_1,lag_2,lag_3,...,Semana_44.0,Semana_45.0,Semana_46.0,Semana_47.0,Semana_48.0,Semana_49.0,Semana_50.0,Semana_51.0,Semana_52.0,Semana_53.0
0,2014,11,0.23,0.97,50.00,103.33,112.12,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2014,89,0.35,0.94,75.67,105.00,112.22,11.00,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2014,127,0.46,0.89,85.25,106.62,112.79,89.00,11.00,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2014,114,0.56,0.83,122.25,106.33,112.34,127.00,89.00,11.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2014,159,0.65,0.76,130.00,107.70,111.93,114.00,127.00,89.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,2024,116,-0.56,0.83,140.00,140.18,142.16,147.00,144.00,156.00,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
569,2024,153,-0.46,0.89,137.00,138.40,141.13,116.00,147.00,144.00,...,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
570,2024,132,-0.35,0.94,135.00,137.44,140.38,153.00,116.00,147.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
571,2024,139,-0.23,0.97,131.00,135.88,141.18,132.00,153.00,116.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


# Dataset Para modelo XGB

In [25]:
dataset = df_encoded

dataset.to_csv('dataset_ml_xgb.csv', index=False)

In [26]:
df_encoded = grouped.drop(["Semana"], axis=1)
df_encoded

,Año,Nuevos_Casos,Semana_sin,Semana_cos,MM_Mensual,MM_Cuarto,MM_Anual,lag_1,lag_2,lag_3
0,2014,11,0.23,0.97,50.00,103.33,112.12,NaN,NaN,NaN
1,2014,89,0.35,0.94,75.67,105.00,112.22,11.00,NaN,NaN
2,2014,127,0.46,0.89,85.25,106.62,112.79,89.00,11.00,NaN
3,2014,114,0.56,0.83,122.25,106.33,112.34,127.00,89.00,11.00
4,2014,159,0.65,0.76,130.00,107.70,111.93,114.00,127.00,89.00
...,...,...,...,...,...,...,...,...,...,...
568,2024,116,-0.56,0.83,140.00,140.18,142.16,147.00,144.00,156.00
569,2024,153,-0.46,0.89,137.00,138.40,141.13,116.00,147.00,144.00
570,2024,132,-0.35,0.94,135.00,137.44,140.38,153.00,116.00,147.00
571,2024,139,-0.23,0.97,131.00,135.88,141.18,132.00,153.00,116.00


# Dataset resultante para los demás modelos

In [27]:
dataset = df_encoded

dataset.to_csv('dataset_ml.csv', index=False)